In [11]:
import pandas 
import spacy 
import scipy 
import sklearn 
import nltk 
import pickle
import time
from sklearn import svm 
from sklearn.svm import LinearSVC 
from scipy import sparse 
from scipy.sparse import csr_matrix
from nltk import corpus
from nltk.corpus import stopwords
from nltk.corpus import wordnet

#start2 = time.clock()

en_nlp = spacy.load("en_core_web_md")

def pre_process(dta): 
    return pandas.get_dummies(dta) 

 
def transform_data_matrix(df_question_train, df_question_predict): 

 
    df_question_train_columns = list(df_question_train.columns) 
    df_question_predict_columns = list(df_question_predict.columns) 

 
    df_question_trans_columns = list(set(df_question_train_columns + df_question_predict_columns)) 

 
    trans_data_train = {} 

 
    for feature in df_question_trans_columns: 
        if feature not in df_question_train: 
            trans_data_train[feature] = [0 for i in range(len(df_question_train.index))] 
        else: 
            trans_data_train[feature] = list(df_question_train[feature]) 

 
    df_question_train = pandas.DataFrame(trans_data_train) 
    df_question_train = csr_matrix(df_question_train) 

 
    trans_data_predict = {} 

 
    for feature in trans_data_train: 
        if feature not in df_question_predict: 
            trans_data_predict[feature] = 0 
        else: 
            trans_data_predict[feature] = list(df_question_predict[feature])   

 
    df_question_predict = pandas.DataFrame(trans_data_predict) 
    df_question_predict = csr_matrix(df_question_predict) 

 
    return df_question_train, df_question_predict 

 
def get_question_predict_data(en_doc): 
    sentence_list = list(en_doc.sents)[0:1] 
    en_nlp = spacy.load("en_core_web_md") 

 
    question_data_frame = [] 

 
    for sentence in sentence_list: 

 
        wh_bi_gram = [] 
        root_token, wh_pos, wh_nbor_pos, wh_word = [""] * 4 
        for token in sentence: 

 
        # if token is of WH question type 
            if token.tag_ == "WDT" or token.tag_ == "WP" or token.tag_ == "WP$" or token.tag_ == "WRB": 
                wh_pos = token.tag_ 
                wh_word = token.text 
                wh_bi_gram.append(token.text) 
                wh_bi_gram.append(str(en_doc[token.i + 1])) 
                wh_nbor_pos = en_doc[token.i + 1].tag_ 
     
            # if token is the root of sentence 
            if token.dep_ == "ROOT": 
                root_token = token.tag_ 
     
        question_data_frame_obj = {'WH': wh_word, 'WH-POS': wh_pos, 'WH-NBOR-POS': wh_nbor_pos, 'Root-POS': root_token} 
        question_data_frame.append(question_data_frame_obj) 
         
        df_question = pandas.DataFrame(question_data_frame) 

 
    return df_question 

 
def support_vector_machine(df_question_train, df_question_class, df_question_predict): 
    lin_clf = LinearSVC() 
    lin_clf.fit(df_question_train, df_question_class) 
    prediction = lin_clf.predict(df_question_predict) 
    return prediction, lin_clf 

 
def classify_question(en_doc): 
    
    f = open('training_data_set.pkl', 'rb')
    df_question_train = pickle.load(f)    
    f.close()
    
    f1 = open('training_data_classes.pkl', 'rb')
    df_question_class = pickle.load(f1)    
    f1.close() 
     
    df_question_predict = get_question_predict_data(en_doc) 
     
    df_question_predict = pre_process(df_question_predict) 
     
    df_question_train, df_question_predict = transform_data_matrix(df_question_train, df_question_predict) 
    predicted_class, svc_clf = support_vector_machine(df_question_train, df_question_class, df_question_predict) 
    return predicted_class 

     
def make_min(sentence,Qtype): 
    sentence = ' '.join([word for word in sentence.split() if word not in (stopwords.words('english') or "?")])
    #start_=process(sentence)
    
    sentenceParts = [] 
     
    if Qtype == "q": 
        for i in range(1,len(sentence.strip("?").split())): 
            sentenceParts.append(sentence.strip("?").split()[i])
        sentence1 = ' '.join([word for word in sentenceParts])
        sentence1 = en_nlp(u'' + sentence1)
        sentenceParts = []
        for token in sentence1:
            sentenceParts.append(token.lemma_)
        
    else: 
        for each in sentence.strip("?").split():
            sentenceParts.append(each)
        sentence1 = ' '.join([word for word in sentenceParts])
        sentence1 = en_nlp(u'' + sentence1)
        sentenceParts = []
        for token in sentence1:
            sentenceParts.append(token.lemma_)
    
    return sentenceParts
  
def findAnAns(question): 
    corpusFile = open("resolved.txt","r") 
    data = corpusFile.read() 
    corpusFile.close() 
 
    data = data.split("." or "!" or "?") 
 
    linedata = [] 
    questionParts = make_min(question,"q") 

    confidenceLevels = [] 
    for sent in data: 
        confidenceLevel = [] 
        matched = 0 
        answerParts = make_min(sent, "a") 
        for each in questionParts: 
            if each in answerParts: 
                matched += 1 
        confidenceLevel.append(sent) 
        confidenceLevel.append(matched*100/len(questionParts)) 
        confidenceLevels.append(confidenceLevel)
    
    v = sorted(confidenceLevels, key=lambda x: x[1], reverse=True)
    #print(v)
    return v
  
def getasentence(sortedconfidencelevels):
    newlist1 = []
    combinedsentence = ''
    newlist2 = []
    sum_ = 0
    for ans in sortedconfidencelevels:
        if ans[1]>60:
            newlist1.append(ans)
    #print(newlist1)
    if len(newlist1)==0:
        return(sortedconfidencelevels[0])
    else:
        for i in newlist1:
            combinedsentence += str(i[0])
            combinedsentence +="."
            sum_ += i[1]
        newlist2 = [combinedsentence, (sum_)/len(newlist1)]
        return newlist2, newlist1
      
def checkifCorrectAnswer(question, answer):
    en_doc_1 = en_nlp(u'' + question)
    en_doc_2 = en_nlp(u'' + answer)
    c = 0
    for token in en_doc_1:
      if token.dep_ == "ROOT":
          target_root = token.text
          target_root = token.lemma_
          #print (target_root)
          syns = wordnet.synsets(target_root)
          synonyms = []
          for i in range(0,len(syns)):
              synonyms.append(syns[i].lemmas()[0].name())
          #print("synonyms",synonyms)
          
    for token in en_doc_2:
        token = token.lemma_
        #print (token)
        if token in synonyms:
            c = 1
           
        
    if c == 0:
        return False
    else:
        return True
    
def get_exact_answer(question, answersent, possiblereplies):
    person_list = []
    reqd_list = []
    entity_list = []
    location_list = []
    number_list =[]
    list_ = []
    en_doc_3 = en_nlp(u'' + answersent)
    en_doc_4 = en_nlp(u''+question)
    if question_class == [u'HUM']:
        for ent in en_doc_3.ents:
            #print(ent.text, ent.label_)
            if ent.label_ == "PERSON" or ent.label_ == "NORP" or ent.label_ =="ORG":
                person_list.append(ent.text)
        reqd_list = person_list
    
    #print(person_list)
    if question_class == [u'NUM']:
        for ent in en_doc_3.ents:
            #print(ent.text, ent.label_)
            if ent.label_ == "DATE" or ent.label_=="PERCENT" or ent.label_=="TIME" or ent.label_=="QUANTITY" or ent.label_=="ORDINAL" or ent.label_=="CARDINAL":
                number_list.append(ent.text)
        reqd_list = number_list
    if question_class == [u"LOC"]:
        for ent in en_doc_3.ents:
            if ent.label_ == "LOC" or ent.label_=="GPE" or ent.label_=="FAC" or ent.label_ == "ORG":
                location_list.append(ent.text)
        reqd_list = location_list
    if question_class == [u'ENTY']:
        with open('picklepath.pkl','rb') as f:
            total_entity_list = pickle.load(f)
            #print (total_entity_list)
        for token in en_doc_3:
            if token.text in total_entity_list:
                entity_list.append(token.text)
        for ent in en_doc_3.ents:
            if ent.text in total_entity_list:
                entity_list.append(ent.text)
        #print(entity_list)
        reqd_list = entity_list
    
    for i in reqd_list:
        for token in en_doc_4:
            if str(i) == token.text:
                reqd_list.remove(i)
                #break
        for ent in en_doc_4.ents:
            #print(ent.text)
            if str(i) == ent.text:
                if i in reqd_list:
                    reqd_list.remove(i)
    print("possible answers:", reqd_list)
    
            
    if len(reqd_list) == 1:
        return reqd_list
    elif len(reqd_list)!=1 and len(reqd_list)!=0:
        for token in en_doc_4:
            if token.tag_ == "WDT" or token.tag_ == "WP" or token.tag_ == "WP$" or token.tag_ == "WRB":
                target_dep = token.dep_
                target_head = token.head.text
        
        for token in en_doc_3:#and token.head.text== target_head:
            #print ("Hey", token.text, token.dep_, token.head.text)
            if token.text in reqd_list:
                #print(token.text)
                if token.dep_ == target_dep:
                    #print(token.text, token.head.text)
                    syns = wordnet.synsets(target_head)
                    synonyms1 = []
                    for i in range(0,len(syns)):
                        synonyms1.append(syns[i].lemmas()[0].name())
                    #print(synonyms1)                
                    if token.head.lemma_ in synonyms1:
                        return token.text       
                    else:
                        return None
    else:
        return None
        
#start1 = time.clock()-start2    
if __name__ == "__main__":
    question = input()
    #start = time.clock()
    question_class = classify_question(en_nlp(u'' + question))
    #print(question_class)
    print("the question type is:", question_class)
    listofsentences = findAnAns(question)
    reply, possiblereplies= getasentence(listofsentences)
    #reply = findAnAns(question)
    #print(reply)
    if reply[1] > 0: 
        checkifCorrectAnswer(question, reply[0])
        if checkifCorrectAnswer(question, reply[0]) == True:
            answer = get_exact_answer(question, reply[0], possiblereplies)
            if answer!=None:
                print("Ans: ", answer)
                print (str(reply[1]) +"% "+ "match")
            else:
                print("Ans text: ", reply[0])
                print (str(reply[1]) +"% "+ "match")
        else:
            print("Only low confidence answer found")
            print("Ans text: ", reply[0])
    else:
        print("invalid question")
    #print("Time taken", time.clock()-start+start1)
                                                                                                                                                                                                                                                                              
    



"Who did Barack Obama campaign against in 2008"
('the question type is:', array([u'HUM'], dtype=object))
Only low confidence answer found
('Ans text: ', ' In 2004 , Barack Obama received national attention during Barack campaign to represent Illinois in the United States Senate with Barack victory in the March Democratic Party primary . Barack began Barack presidential campaign in 2007 and , after a close primary campaign against Hillary Rodham Clinton in 2008 , Barack won sufficient delegates in the Democratic Party primaries to receive the presidential nomination .')
